#### Import Julia packages

In [1]:
# import Pkg
# Pkg.add("JSON3")
# Pkg.add("Interpolations")

In [2]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Interpolations

#### Import Custom functions

In [11]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

#### Read JSON File

In [4]:
width, height, windows, doors, heating_elem = get_room("Boundary.json")

(5, 3, Any[(0.75125, 2.32125, "Left"), (0.73125, 2.32125, "Right")], Any[(0.235, 0.655, "Top")], [[2.005, 3.435, 0.12125, 0.20125], [1.765, 3.215, 2.82125, 2.89125]])

In [5]:
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

(Any[0.01669, 0.01319, 0.00967, 0.01037, 0.02006, 0.02314, 0.01764, 0.01828, 0.01869, 0.01863, 0.01817, 0.01777], Any[0.19048, 0.15054, 0.11036, 0.11835, 0.22894, 0.2641, 0.20133, 0.20863, 0.21331, 0.21262, 0.20737, 0.20281], Any[282.15, 282.34999999999997, 282.25, 282.34999999999997, 282.15, 281.45, 280.65, 280.84999999999997, 280.75, 280.45, 280.25, 279.95], Any[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

#### Boundary conditions

In [6]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [7]:
domain = (0, width , 0, height)
partition = (25,25)
model = CartesianDiscreteModel(domain,partition) 
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) 
# display(fig)

UnconstrainedFESpace()

#### Time parameters

In [8]:
t0 = 0.0    # Start time
tF = 10.0   # End time
Δt = 0.05   # Timestep

0.05

#### Room parameters

In [10]:
timescale = 500
h_wall(x) = 0.22*timescale            # [W/m^2K]
h_window(x) = 1.2*timescale
h_door(x) = 1.0*timescale

ρ(x)=1.225
c(x)=1020.0
k(x)=15*timescale

h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 

#Toutdoor(x,t)=273.0
#Tout(t)=x->Toutdoor(x,t)

Tout = get_temp_function(T_out, tF)

price = get_price_function(price_NOK, tF, 1/timescale*0.1)

Tini(x)=297.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(297.0, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 625
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 1039127851119791941

#### Controll parameter

In [12]:
#q_pos(x) = χ(x[1], 0.21, 1.73) * χ(x[2], 0.13, 0.16)
q_pos = heating_elements(heating_elem)
Q(x,t)=χ(t,0.0,100.0)*3000.0*q_pos(x)*0.0
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [13]:
L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)

γ = 5000.0

Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=1000.0*timescale
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

Proj (generic function with 2 methods)

#### Test Gradient DEs

In [14]:
(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=25,
armijoparas=(ρ=0.5, α_0=1, α_min=1e-5, σ=1e-4), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=1.0396198093705969e7, k = 1
α = 1, new_cost = 1.0396198093705969e7
entered for loop, E=8.166685714944502e6, k = 2
α = 1, new_cost = 8.166685714944502e6
entered for loop, E=7.244755563787114e6, k = 3
α = 1, new_cost = 7.244755563787114e6
entered for loop, E=6.701219864281684e6, k = 4
α = 1, new_cost = 6.701219864281684e6
entered for loop, E=6.36503824423994e6, k = 5
α = 1, new_cost = 6.36503824423994e6
entered for loop, E=6.162570558050462e6, k = 6
α = 1, new_cost = 6.162570558050462e6
entered for loop, E=6.05130610483729e6, k = 7
α = 1, new_cost = 6.05130610483729e6
entered for loop, E=6.003488449730772e6, k = 8
α = 1, new_cost = 6.003488449730772e6
entered for loop, E=5.992351785547102e6, k = 9
α = 1, new_cost = 5.992351785547102e6
entered for loop, E=5.98573784418133e6, k = 10
α = 1, new_cost = 5.98573784418133e6
entered for loop, E=5.984093061570973e6, k = 11
α = 1, new_cost = 5.984093061570973e6
α = 0.5, new_cost = 5.984093061570973e6
α = 0

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (0.05, SingleFieldFEFunction()), (0.1, SingleFieldFEFunction()), (0.15000000000000002, SingleFieldFEFunction()), (0.2, SingleFieldFEFunction()), (0.25, SingleFieldFEFunction()), (0.3, SingleFieldFEFunction()), (0.35, SingleFieldFEFunction()), (0.39999999999999997, SingleFieldFEFunction()), (0.44999999999999996, SingleFieldFEFunction())  …  (9.55, SingleFieldFEFunction()), (9.600000000000001, SingleFieldFEFunction()), (9.650000000000002, SingleFieldFEFunction()), (9.700000000000003, SingleFieldFEFunction()), (9.750000000000004, SingleFieldFEFunction()), (9.800000000000004, SingleFieldFEFunction()), (9.850000000000005, SingleFieldFEFunction()), (9.900000000000006, SingleFieldFEFunction()), (9.950000000000006, SingleFieldFEFunction()), (10.000000000000007, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction())

In [15]:
# Opprett en mappe for midlertidige filer hvis den ikke eksisterer
if !isdir("tmp")
    mkdir("tmp")
end

# Initialtilstanden og resten av tidsløsningene
# uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# uh  = solution[2:end]  # De resterende løsningene


uh = Ts
# Lagre resultater i Paraview-format (VTK)
createpvd("results") do pvd
    # Første løsning (t=0)
    # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

    # Lagrer løsninger for hver tidssteg
    for (tn, uhn) in uh
        pvd[tn] = createvtk(Ω, "tmp/results_$tn.vtu", cellfields=["u" => uhn])
    end
end


202-element Vector{String}:
 "results.pvd"
 "tmp/results_0.0.vtu"
 "tmp/results_0.05.vtu"
 "tmp/results_0.1.vtu"
 "tmp/results_0.15000000000000002.vtu"
 "tmp/results_0.2.vtu"
 "tmp/results_0.25.vtu"
 "tmp/results_0.3.vtu"
 "tmp/results_0.35.vtu"
 "tmp/results_0.39999999999999997.vtu"
 ⋮
 "tmp/results_9.600000000000001.vtu"
 "tmp/results_9.650000000000002.vtu"
 "tmp/results_9.700000000000003.vtu"
 "tmp/results_9.750000000000004.vtu"
 "tmp/results_9.800000000000004.vtu"
 "tmp/results_9.850000000000005.vtu"
 "tmp/results_9.900000000000006.vtu"
 "tmp/results_9.950000000000006.vtu"
 "tmp/results_10.000000000000007.vtu"

In [16]:
T = get_temperature_int(Ts, width, height)
q = get_control_int(qs)
result_plot()

sys:1: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
